In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:

df = pd.DataFrame(p.dict() for p in spotprice)
df = df.loc[lambda x: x.PriceArea.isin(["DK1", "DK2"])]
df = df.reset_index()

df_day = df.groupby('PriceArea').resample(rule='1D', on='HourUTC').SpotPriceDKK.mean().reset_index()
df_month = df.groupby('PriceArea').resample(rule='1M', on='HourUTC').SpotPriceDKK.mean().reset_index()
df_month['HourUTC'] = df_month['HourUTC'].dt.strftime('%Y-%m')

fig, ax = plt.subplots()
sns.lineplot(
    x='HourUTC', 
    y='SpotPriceDKK', 
    hue='PriceArea', 
    data=df_day,
    ax=ax
)
fig, ax = plt.subplots()
sns.lineplot(
    x='HourUTC', 
    y='SpotPriceDKK', 
    hue='PriceArea', 
    data=df_month,
    ax=ax
)
plt.show()

print(df)

